# SI 106 Problem Set 8

Primary Topics:
- Internet APIs I
- Internet APIs II

Lookahead Topics:
- Exceptions

---
---

To avoid problems with rate limits and site accessibility, we have provided a cache file with results for all the queries you need to make . Just use ``requests_with_caching.get()`` rather than ``requests.get()``-- as described in the textbook, they both take the same parameters and return the same kinds of `Response` objects. 

If you're having trouble, you may not be formatting your queries properly, or you may not be asking for data that exists in our cache. We will try to provide as much information as we can to help guide you to form queries for which data exists in the cache.

### TasteDive
Your first task will be to fetch data from TasteDive. The documentation for the API is at https://tastedive.com/read/api.

In [18]:
import requests_with_caching

---
## Problem 1
Define a function, called `get_movies_from_tastedive`. It should take one input parameter, a string that is the name of a movie or music artist. The function should return the 5 TasteDive results that are associated with that string; be sure to only get movies, not other kinds of media. It will be a python dictionary with just one key, 'Similar'.

Try invoking your function with the input "Black Panther".

HINT: Be sure to include **only** `q`, `type`, and `limit` as parameters in order to extract data from the cache. If any other parameters are included, then the function will not be able to recognize the data that you're attempting to pull from the cache. 

You will *not* need an api key in order to complete the project, because all data will be found in the cache. If you ever get a result indicating that the result was not found in the permanent cache, it means you haven't formulated the query quite right. If you do provide a tastedive API key, your request may succeed even when the data is not found in cache. That may be useful to you in your explorations, but you will not pass the tests for the problem set unless you find the data in the permanent cache.

    The cache includes data for the following queries:

        =============== ==========  ==========
          q             type        limit
        =============== ==========  ==========
        Black Panther   <omitted>   <omitted>
        Black Panther   <omitted>   5
        Black Panther   movies      <omitted>
        Black Panther   movies      5
        Tony Bennett    <omitted>   5
        Tony Bennett    movies      5
        Captain Marvel  movies      5
        Bridesmaids     movies      5
        Sherlock Holmes movies      5
        =============== ==========  ==========

In [19]:
import json

def get_movies_from_tastedive(movie):
    param = {"q": movie, "type": "movies", "limit": 5}
    td_response = requests_with_caching.get("https://tastedive.com/api/similar", params = param)
    py_data = json.loads(td_response.text)
    return py_data

get_movies_from_tastedive("Black Panther")

found in permanent_cache


{'Similar': {'Info': [{'Name': 'Black Panther', 'Type': 'movie'}],
  'Results': [{'Name': 'Captain Marvel', 'Type': 'movie'},
   {'Name': 'Avengers: Infinity War', 'Type': 'movie'},
   {'Name': 'Ant-Man And The Wasp', 'Type': 'movie'},
   {'Name': 'The Fate Of The Furious', 'Type': 'movie'},
   {'Name': 'Deadpool 2', 'Type': 'movie'}]}}

### Problem 1 Tests:

In [20]:
# sample invocation
get_movies_from_tastedive("Black Panther")

assert type(get_movies_from_tastedive("Black Panther")) == dict

assert get_movies_from_tastedive("Bridesmaids")['Similar']["Info"][0]["Name"] == "Bridesmaids"

assert len(get_movies_from_tastedive("Black Panther")['Similar']['Results']) == 5

# check that only movies are returned unless we ask for something else
results = get_movies_from_tastedive("Tony Bennett")
print(results)
non_movies = [x for x in results['Similar']["Results"] if x['Type'] != "movie"]
assert non_movies == []

found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
{'Similar': {'Info': [{'Name': 'Tony Bennett', 'Type': 'music'}], 'Results': [{'Name': 'The Startup Kids', 'Type': 'movie'}, {'Name': 'Charlie Chaplin', 'Type': 'movie'}, {'Name': 'Venus In Fur', 'Type': 'movie'}, {'Name': 'Loving', 'Type': 'movie'}, {'Name': 'The African Queen', 'Type': 'movie'}]}}


---
## Problem 2
Next, you will need to write a function that extracts movie titles. Call it ``extract_movie_titles``. It takes as input a dictionary that was returned by ``get_movies_from_tastedive``. ``extract_movie_titles`` should return a list containing just the titles of the movies.

In [21]:
def extract_movie_titles(get_mov_dict):
    mov_title = []
    mov_info = get_mov_dict["Similar"]["Results"]
    for mov in mov_info:
        mov_title.append(mov["Name"])
    return mov_title

### Problem 2 Tests:

In [22]:
extract_movie_titles(get_movies_from_tastedive("Tony Bennett"))

assert extract_movie_titles(get_movies_from_tastedive("Tony Bennett")) == ['The Startup Kids', 'Charlie Chaplin', 'Venus In Fur', 'Loving', 'The African Queen']

found in permanent_cache
found in permanent_cache


---
## Problem 3
Next, you'll write a function, called ``get_related_titles``. It takes *a list of movie titles* as input. It gets five related movies for each from TasteDive, extracts the titles for all of them, and combines them all into a single list. Don't include the same movie twice.

In [23]:
def get_related_titles(mov_lst):
    new_lst = []
    for title in mov_lst:
        x = get_movies_from_tastedive(title)
        y = extract_movie_titles(x)
        for movie in y:
            if movie not in new_lst:
                new_lst.append(movie)
    return new_lst

### Problem 3 Tests:

In [24]:
get_related_titles(["Black Panther", "Captain Marvel"])

found in permanent_cache
found in permanent_cache


['Captain Marvel',
 'Avengers: Infinity War',
 'Ant-Man And The Wasp',
 'The Fate Of The Furious',
 'Deadpool 2',
 'Inhumans',
 'Venom',
 'American Assassin',
 'Black Panther']

In [25]:
assert get_related_titles([]) == []

In [26]:
assert get_related_titles(["Black Panther", "Captain Marvel"]) == ['Captain Marvel', 'Avengers: Infinity War', 'Ant-Man And The Wasp', 'The Fate Of The Furious', 'Deadpool 2', 'Inhumans', 'Venom', 'American Assassin', 'Black Panther']

found in permanent_cache
found in permanent_cache


## OMDB

---
## Problem 4: OMDB `get_movie_data`
Your next task will be to fetch data from OMDB. The documentation for the API is at https://www.omdbapi.com/

Define a function called ``get_movie_data``. It takes in one parameter which is a string that should represent the title of a movie you want to search. The function should return a dictionary with information about that movie.

Again, use ``requests_with_caching.get()``. For the queries on movies that are already in the cache, you *won't* need an api key. You will need to provide the following keys: ``t`` and ``r``. As with the TasteDive cache, be sure to **only** include those two parameters in order to extract existing data from the cache.

In [27]:
def get_movie_data(mov_title):
    param = {"t": mov_title, "r": "json"}
    omdb_response = requests_with_caching.get("http://www.omdbapi.com/", params=param)
    info = json.loads(omdb_response.text)
    return info

### Problem 4 Tests

In [28]:
get_movie_data("Venom")

assert type(get_movie_data("Venom")) == dict

assert get_movie_data("Baby Mama")["Title"] == "Baby Mama"
print("http://www.ombdapi.com/?apikey=5df43a8e&t=Bridesmaids&r=json")
get_movie_data("Bridesmaids")

found in permanent_cache
found in permanent_cache
found in permanent_cache
http://www.ombdapi.com/?apikey=5df43a8e&t=Bridesmaids&r=json
found in page-specific cache


{'Response': 'False', 'Error': 'No API key provided.'}

---
## Problem 5: OMDB `get_movie_rating`
Now write a function called ``get_movie_rating``. It takes an OMDB dictionary result for one movie and extracts the Rotten Tomatoes rating as an integer. For example, if given the OMDB dictionary for `"Black Panther"`, it would return `97`. If there is no Rotten Tomatoes rating, return `0`.

In [29]:
def get_movie_rating(omdb_dict):
    strRating=""
    for typeRatingList in omdb_dict["Ratings"]:
        if typeRatingList["Source"]== "Rotten Tomatoes":
            strRating = typeRatingList["Value"]
    if strRating != "":
        rating = int(strRating[:2])
    else:
        rating = 0
    return rating

### Problem 5 Tests:

In [30]:
get_movie_rating(get_movie_data("Deadpool 2"))

assert type(get_movie_rating(get_movie_data("Deadpool 2"))) == int

assert get_movie_rating(get_movie_data("Venom")) == 0

assert get_movie_rating(get_movie_data("Deadpool 2")) == 83

found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache


---
## Problem 6: OMDB `get_sorted_recommendations`

Now, you'll put it all together. Define a function ``get_sorted_recommendations``. It takes a list of movie titles as an input. It returns a sorted list of related movie titles as output, up to five related movies for each input movie title. The movies should be sorted in descending order by their Rotten Tomatoes rating, as returned by the ``get_movie_rating`` function. Break ties in reverse alphabetic order, so that `'Yahşi Batı'` comes before `'Eyyvah Eyvah'`.

In [31]:
def get_sorted_recommendations(movie_titles_lst):
    list_movie= get_related_titles(movie_titles_lst)
    list_movie= sorted(list_movie, key = lambda movie_name: (get_movie_rating(get_movie_data(movie_name)), movie_name), reverse=True)
    
    return list_movie

#### Problem 6 Tests

In [32]:
sample_actual_recommendations = get_sorted_recommendations(["Bridesmaids", "Sherlock Holmes"])
assert type(sample_actual_recommendations) == list
assert sample_actual_recommendations == ['Date Night', 'The Heat', 'Baby Mama', 'The Five-Year Engagement', 'Sherlock Holmes: A Game Of Shadows', 'Bachelorette', 'Prince Of Persia: The Sands Of Time', 'Pirates Of The Caribbean: On Stranger Tides', 'Yahşi Batı', 'Eyyvah Eyvah']

found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache
found in permanent_cache


---
## Problem 7 (Demonstrate Your Understanding)

For this problem, you will show that you understand how to look at API documentation and write code to reference an API. Look at [this list of public APIs](https://github.com/public-apis/public-apis). Select an API **that we have not used in problem sets, discussion sections, or the textbook** where "Auth" is either "No" or "`apiKey`" and write code that references that API. Your code should pass in at least one variable parameter to the API (meaning that you should select an API that accepts parameters).

---
## Problem 8 (Look Ahead to Exceptions)

In plain English, explain what the `getInteger` function below does. What is the role of the `try` and `except` blocks?

```python
def getInteger(low=0, high=100):
    while True:
        user_inp = input('Enter a number between {} and {}'.format(low, high))
        try:
            val = int(user_inp)
        except ValueError:
            print('Enter an integer!')
            continue
        if val >= low and val <= high:
            return val
```

It will keep asking for a number between 0 and 100 (defaultly, you can change it using optional paramaters while invoking the function) until you input one and then returns that value. Try is used to run the function and except is used to deal with the situation, when there's a ValueError (e.g. you write a string). If you write e.g. the string, the function will print an Error message, asking for an integer and will run the function again.

---
---
## Collaborators and Sources of Help
In this markdown cell, list names of any people you worked with or got help from, and provide links to or descriptions of any external resources you consulted.

- List your collaborators here

## Upload your solutions to Canvas

Upload your .ipynb file to Canvas.

*Note: You may submit any number of times before the deadline*


    MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM
    MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM
    MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM
    MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM
    MMMMMMMM             MMMMMMMMMMMMMMMMM             MMMMMMMMM
    MMMMMMMM              MMMMMMMMMMMMMMM              MMMMMMMMM
    MMMMMMMM                MMMMMMMMMMM                MMMMMMMMM
    MMMMMMMM                 MMMMMMMMM                 MMMMMMMMM
    MMMMMMMM                  MMMMMMM                  MMMMMMMMM
    MMMMMMMMMMMM               MMMMM                MMMMMMMMMMMM
    MMMMMMMMMMMM                MMM                 MMMMMMMMMMMM
    MMMMMMMMMMMM                 V                  MMMMMMMMMMMM
    MMMMMMMMMMMM                                    MMMMMMMMMMMM
    MMMMMMMMMMMM         ^               ^          MMMMMMMMMMMM
    MMMMMMMMMMMM         MM             MM          MMMMMMMMMMMM
    MMMMMMMMMMMM         MMMM         MMMM          MMMMMMMMMMMM
    MMMMMMMMMMMM         MMMMM       MMMMM          MMMMMMMMMMMM
    MMMMMMMMMMMM         MMMMMM     MMMMMM          MMMMMMMMMMMM
    MMMMMMMM                MMMM   MMMM                MMMMMMMMM
    MMMMMMMM                MMMMMVMMMMM                MMMMMMMMM
    MMMMMMMM                MMMMMMMMMMM                MMMMMMMMM
    MMMMMMMM                MMMMMMMMMMM                MMMMMMMMM
    MMMMMMMM                MMMMMMMMMMM                MMMMMMMMM
    MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM
    MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM
    MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM
    MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM
    MMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM  